<a href="https://colab.research.google.com/github/AdityaPrabha/Applied-Data-Science-with-Python/blob/master/RedCarpet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name - Aditya Prabha                       


Email-id - prabhaaditya37@gmail.com


Github link - https://github.com/AdityaPrabha


In [0]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import Legend
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns;sns.set()
from datetime import date
from nsepy import get_history
from sklearn import metrics

output_notebooks()

In [0]:
infy = get_history(symbol="INFY",
                  start = date(2015,1,1),
                  end = date(2016,12,31))

tcs = get_history(symbol="TCS", 
                 start = date(2015,1,1),
                 end = date(2016,12,31))


#tcs = pd.read_csv("C:/Users/Cartan/tcs.csv")
#infy = pd.read_csv("C:/Users/Cartan/infy.csv")
tcs.reset_index(level=0, inplace=True)
infy.reset_index(level=0, inplace=True)


In [0]:
tcs.head()

In [0]:
infy.head()

In [0]:
tcs.info()

In [0]:
infy.info()

__Part 1:__


__Create 4,16,....,52 week moving average(closing price) for each stock and index. This should happen through a function.__

In [0]:
def movingAverage(Stock,N):
    cumsum, moving_aves = [0], []
    for i, x in enumerate(Stock, 1):
        cumsum.append(cumsum[i-1] + x)
        if i>=N:
            moving_ave = (cumsum[i] - cumsum[i-N])/N
            moving_aves.append(moving_ave)
    return(moving_aves)
    print(moving_aves)

In [0]:
k = infy["Close"]                                     # where k is stock name (INFY ,TCS)
N = 52                                                 # where N is equal to no. of weeks
m= (N*7)
p = movingAverage(k,m)
p = pd.DataFrame(p)
p

3.1. Volume Shocks: If the volume traded is 10% higher/lower than the previous day - make a 0/1 boolean time series for shock, 0/1 dummy-coded time series for direction of shock.

3.2. Price Shocks: If closing price at T vs T+1 has a difference > 2%, then 0/1 boolean time series for shock, 0/1 dummy-coded time series for direction of shock.

**INFY**

In [0]:
# closing price of INFY at T vs T+1 has a difference > 2%
infy["Close_diff > 2%"] = (infy['Close'] - infy['Close'].shift(1))>(0.02 * infy["Close"])
dummy_c = pd.get_dummies(infy["Close_diff > 2%"])
dummy_c = pd.DataFrame(dummy_c)

# volume traded of INFY is 10% higher/lower than previous day
infy['Volume_diff> 10%'] = (infy['Volume'] - infy['Volume'].shift(1))>(0.1 * infy["Volume"])
dummy_v = pd.get_dummies(infy['Volume_diff> 10%'])
dummy_v = pd.DataFrame(dummy_v)


ic = pd.concat([infy["Close"],infy["Close_diff > 2%"],dummy_c], axis=1)
icc = pd.concat([infy["Date"],infy["Symbol"], ic],axis=1)
iv = pd.concat([infy["Volume"],infy["Volume_diff> 10%"],dummy_v], axis=1)
infy_dummy = pd.concat([icc,iv], axis=1)

In [0]:
infy_dummy.head()

**TCS**

In [0]:
# closing price of TCS at T vs T+1 has a difference > 2%
tcs["Close_diff > 2%"] = (tcs['Close'] - tcs['Close'].shift(1))>(0.02 * tcs["Close"])
dummy_c = pd.get_dummies(tcs["Close_diff > 2%"])
dummy_c = pd.DataFrame(dummy_c)

# volume traded of TCS is 10% higher/lower than previous day
tcs['Volume_diff> 10%'] = (tcs['Volume'] - tcs['Volume'].shift(1))>(0.1 * tcs["Volume"])
dummy_v = pd.get_dummies(tcs['Volume_diff> 10%'])
dummy_v = pd.DataFrame(dummy_v)

tc = pd.concat([tcs["Close"],tcs["Close_diff > 2%"],dummy_c], axis=1)
tcc = pd.concat([tcs["Date"],tcs["Symbol"], tc],axis=1)
tv = pd.concat([tcs["Volume"],tcs["Volume_diff> 10%"],dummy_v], axis=1)
tcs_dummy = pd.concat([tcc,tv], axis=1)

In [0]:
tcs_dummy.head()

**Part 2: Data Visualization**

In [0]:
fig = figure(plot_width=800, plot_height=400)



X = infy['Date']
Y = infy['Close']

fig.line(X,Y, line_width=2, legend='INFY Close')

show(fig)

In [0]:
fig = figure(plot_width=800, plot_height=400)



X = tcs['Date']
Y = tcs['Close']
fig.line(X,Y, line_width=2, legend='TCS Close')

show(fig)

In [0]:
fig1 = figure(plot_width=800, plot_height=400, title = 'infy and tcs')

fig1.multi_line([infy['Date'],infy['Date']], [infy['Close'],tcs['Close']],
             color=["blue", "red"], alpha=[0.8, 0.8], line_width=2)

show(fig1)

In [0]:
movingAverage1 = infy["Close"].rolling(window=10).mean()
movingSTD1 = infy["Close"].rolling(window=10).std()

movingAverage2 = infy["Close"].rolling(window=75).mean()
movingSTD2 = infy["Close"].rolling(window=75).std()

fig1 = figure(plot_width=800, plot_height=400)

fig1.multi_line([infy['Date'],infy['Date'],infy['Date']], [infy['Close'], movingAverage1, movingSTD1],
             color=["blue", "red", "black"], alpha=[0.8, 0.8, 0.8], line_width=2)
show(fig1)

plt.figure(figsize=(20,7))
fig2 = figure(plot_width=800, plot_height=400)

fig2.multi_line([infy['Date'],infy['Date'],infy['Date']], [infy['Close'], movingAverage2, movingSTD2],
             color=["blue", "red", "black"], alpha=[0.8, 0.8, 0.8], line_width=2)
show(fig2)

In [0]:
movingAverage1 = tcs["Close"].rolling(window=10).mean()
movingSTD1 = tcs["Close"].rolling(window=10).std()

movingAverage2 = tcs["Close"].rolling(window=75).mean()
movingSTD2 = tcs["Close"].rolling(window=75).std()

fig1 = figure(plot_width=800, plot_height=400)

fig1.multi_line([tcs['Date'],tcs['Date'],tcs['Date']], [tcs['Close'], movingAverage1, movingSTD1],
             color=["blue", "red", "black"], alpha=[0.8, 0.8, 0.8], line_width=2)
show(fig1)

plt.figure(figsize=(20,7))
fig2 = figure(plot_width=800, plot_height=400)

fig2.multi_line([tcs['Date'],tcs['Date'],tcs['Date']], [tcs['Close'], movingAverage2, movingSTD2],
             color=["blue", "red", "black"], alpha=[0.8, 0.8, 0.8], line_width=2)
show(fig2)

**Partial Autocorrelation Factor (PACF) & Autocorrelation Factor (ACF)**

**TCS**

In [0]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    # Determining rolling statistics
    movingAverage = timeseries.rolling(window = 52).mean()
    movingSTD = timeseries.rolling(window = 52).std()
    
    # Plot rolling statistics
    plt.figure(figsize=(20,7))
    orig = plt.plot(timeseries, color = 'blue', label = 'Original')
    mean = plt.plot(movingAverage, color = 'red', label = 'Rolling Mean')
    std = plt.plot(movingSTD, color = 'black', label = 'Rolling Std')
    plt.legend(loc = 'best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block = False)
    
    # Perform Dickey-Fuller Test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag = 'AIC')
    dfoutput = pd.Series(dftest[0:4], index = ['Test Statistics', 'p-value', '#Lags Used', 'Number of Observation Used'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [0]:
test_stationarity(tcs["Close"])

**Reject null hypothesis, since p-value < 1, and critical value greater than test statistics**

In [0]:
dataset_log = np.log(tcs["Close"])

In [0]:
test_stationarity(dataset_log)

In [0]:
#  PACF plots

from statsmodels.tsa.stattools import pacf
def Pacf(dataset_log):
    
    dataset_pacf = pacf(dataset_log, nlags = 20, method = 'ols')

    plt.figure(figsize=(20,7))

    # Plot PACF:


    '''plt.subplot(122)
    plt.plot(dataset_pacf)
    plt.axhline(y = 0, linestyle = '--', color = 'gray')
    plt.axhline(y = -1.96 / np.sqrt(len(dataset_log)), linestyle = '--', color = 'gray')
    plt.axhline(y = 1.96/np.sqrt(len(dataset_log)), linestyle = '--', color = 'gray')
    plt.title('Partial Autocorrelation Function')
    plt.tight_layout()
       
    print(dataset_pacf)
    '''
    x = np.linspace(0,len(dataset_pacf),len(dataset_pacf))
    y = [0*i for i in x]
    y1 = [-1.96 / np.sqrt(len(dataset_log)) for i in range(len(dataset_pacf))]
    y2 = [+1.96 / np.sqrt(len(dataset_log)) for i in range(len(dataset_pacf))]
    fig1 = figure(plot_width=800, plot_height=400)

    fig1.multi_line([x,x,x,x], [dataset_pacf,y,y1,y2],
             color=["blue", "grey", "grey", "grey"], alpha=[0.8, 0.8, 0.8, 0.8], line_width=2)
    show(fig1)

In [0]:
Pacf(dataset_log)

**INFY**

In [0]:
test_stationarity(infy["Close"])

In [0]:
dataset_log = np.log(infy["Close"])

In [0]:
test_stationarity(dataset_log)

In [0]:
Pacf(dataset_log)

**Part 3: Data Modeling**

**Linear Model**

In [0]:
feature_cols = ['Prev Close', 'Open']            
X = infy[feature_cols]     
y = infy["Close"]

In [0]:
def linear_reg( X, y, RandomizedSearchCV = False):
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state=1) 
    
    from sklearn.linear_model import LinearRegression
    linreg = LinearRegression()
    
    if not(RandomizedSearchCV):
        linreg.fit(X_train, y_train) 
        
        feature_cols.insert(0,'Intercept')
        coef = linreg.coef_.tolist()
        coef.insert(0, linreg.intercept_)
        
        eq1 = zip(feature_cols, coef)
        for c1,c2 in eq1:
            print(c1,":-",c2)
        
        # Root Mean Squared Error (RMSE) is the square root of the mean of the squared errors:
        y_pred_train = linreg.predict(X_train)  
        y_pred_test = linreg.predict(X_test)     
        
        RMSE_train = np.sqrt( metrics.mean_squared_error(y_train, y_pred_train))
        RMSE_test = np.sqrt(metrics.mean_squared_error(y_test, y_pred_test))
        print('RMSE for training set is {}'.format(RMSE_train))
        print('RMSE for test set is {}'.format(RMSE_test))
    
        # R-Squared and Ajdusted R-Squared of Training

        yhat = linreg.predict(X_train)
        SS_Residual = sum((y_train-yhat)**2)
        SS_Total = sum((y_train-np.mean(y_train))**2)
        r_squared = 1 - (float(SS_Residual))/SS_Total
        adjusted_r_squared = 1 - (1-r_squared)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
        print("R-Squared of Training = ",r_squared)
        print("Adjusted R-Squared of Training = ", adjusted_r_squared)
        
        
        # R-Squared and Ajdusted R-Squared of Testing

        yhat = linreg.predict(X_test)
        SS_Residual = sum((y_test-yhat)**2)
        SS_Total = sum((y_test-np.mean(y_test))**2)
        r_squared = 1 - (float(SS_Residual))/SS_Total
        adjusted_r_squared = 1 - (1-r_squared)*(len(y_test)-1)/(len(y_test)-X_test.shape[1]-1)
        print("R-Squared of Testing = ",r_squared)
        print("Adjusted R-Squared of Testing = ",adjusted_r_squared)
        
        
    else:
        from sklearn.model_selection import RandomizedSearchCV
        parameters = {'normalize':[True,False], 'copy_X':[True, False]}
        linreg = RandomizedSearchCV(linreg,parameters,n_iter=10, cv = None)
        linreg.fit(X_train, y_train)                                       
        print("Mean cross-validated score of the best_estimator : ", linreg.best_score_)  
        
        y_pred_test = linreg.predict(X_test)                                               

        RMSE_test = (metrics.mean_squared_error(y_test, y_pred_test))             
        print('RMSE for the test set is {}'.format(RMSE_test))

    return(linreg)

**INFY**

**Without Grid Search**

In [0]:
linreg = linear_reg(X, y)

**Applying Grid Search method for exhaustive search over specified parameter values of the estimator**

In [0]:
linreg=linear_reg(X,y, RandomizedSearchCV = True)

**TCS**

In [0]:
feature_cols = ['Prev Close', 'Open']
X = tcs[feature_cols]     
y = tcs["Close"]

**Without Grid Search**

In [0]:
linreg = linear_reg(X, y)

**Applying Grid Search method for exhaustive search over specified parameter values of the estimator**

In [0]:
linreg=linear_reg(X,y, RandomizedSearchCV = True)